# Preparation
Import libraries

In [ ]:
import gym # openAi gym
from gym import envs
import numpy as np 
import datetime
import keras 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import pandas as pd 
from time import sleep

Create a [Taxi](https://gym.openai.com/envs/Taxi-v3/) game environment and run it 

In [ ]:
env = gym.make('Taxi-v3')
env.reset()
env.render()

**Legend:**

| --> Wall (can't cross or change positions)

Yellow --> Taxi's current location

Blue --> Pick-up location

Purple --> Drop-off location

Green --> Taxi turns green once it picks up a passenger

Letters --> Locations

In [ ]:
env.action_space.n

6

**6 actions:**
*   0: move south
*   1: move north
*   2: move east
*   3: move west
*   pickup passenger
*   dropoff passenger

Just like other games implemeted in Gym, Taxi specifies **rewards**:
+20: last step when a taxi picked up a passenger and droped them off at the target location
-1: for each step the agent takes to find the optimal solution 
-10: every time a passenger is picked up or dropped off incorrectly

# Tasks

Start by implementing a random search approach. The agent performs random actions until the task is accomplished (the passenger is successfully picked up and dropped off)

In [ ]:
# your code:
penalties, reward = 0, 0
env.reset()
env = env.unwrapped # https://github.com/openai/gym/issues/336 removes 200 steps timelimit
for t in range(20000):
 env.render()
 action = env.action_space.sample()
 observation, reward, done, info = env.step(action)
 print(observation)
 print(reward)
 print(done)
 print(info)
 if reward == -10:
  penalties += 1
 if done:
  print("Episode finished after {} timesteps".format(t+1))
  print('number of penalties:', penalties)
  break

env.close()

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
+---------+
  (West)
44
-1
False
{'prob': 1.0}
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
64
-1
False
{'prob': 1.0}
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
64
-10
False
{'prob': 1.0}
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)
64
-10
False
{'prob': 1.0}
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)
64
-1
False
{'prob': 1.0}
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
44
-1
False
{'prob': 1.0}
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
44
-10
False
{'prob': 1.0}
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)
44
-10
False
{'prob': 1.0}
+---------+
|R: | : :G|
| : | : : |
| : :

Get a feeling of how efficient this approach is. Run the model 1000 times and get the average amount of steps the agent took to complete the mission.

In [ ]:
# your code:
average_timesteps = []
average_penalties = []
for _ in range(1000):   
 penalties, reward = 0, 0
 env.reset()
 env = env.unwrapped # https://github.com/openai/gym/issues/336 removes 200 steps timelimit
 for t in range(20000):
  action = env.action_space.sample()
  observation, reward, done, info = env.step(action)
  if reward == -10:
   penalties += 1
  if done:
   #print("Episode finished after {} timesteps".format(t+1))
   #print('number of penalties:', penalties)
   t+1
   average_timesteps.append(t)
   average_penalties.append(penalties)
   break
 env.close()





In [ ]:
def Average(lst):
    return sum(lst) / len(lst)
print('average steps: ', Average(average_timesteps))
print('average penalties: ', Average(average_penalties))

average steps:  2510.941
average penalties:  812.179


In the following task, we will attempt to make the random algorithm smarter by introducting the Q-Learning approach with epsylon-greedy method.

The algorithm needs to function as follows:

*   Initialize a Q-table with zeros only.
*   For each state, select any of the possible actions for the current state (S)
*   Travel to the next state (S') as a result of that action (a).
*   For all possible actions from the state (S'), select the one with the highest Q-value with probability (1 - epsilon) and select a random action with probability epsilon. This is to balance the exploration and exploitation actions.
*   Update Q-table values using the equation: Q(state, action) <- (1−α) Q(state, action) + α(reward + γ max_α_Q(next state, all actions))
*   Set the next state as the current state.
*   If the goal state is reached, end and repeat the process.

In [ ]:
import random
from IPython.display import clear_output
import numpy as np

def Q_learning_train(env,alpha,gamma,epsilon,episodes): 
    """Q Learning Algorithm with epsilon greedy 

    Args:
        env: Environment 
        alpha: Learning Rate --> extent to which the Q-values are being updated in every iteration.
        gamma: Discount Rate --> how much importance is given to future rewards
        epsilon: probability of selecting random action vs. the 'optimal' action
        episodes: number of episodes to train on

    Returns:
        Q-learning Trained policy

    """
    %%time
    """Training the agent"""

    # your code:

    q_table = np.zeros([env.observation_space.n, env.action_space.n])
    
    for i in range(episodes):
     state = env.reset()
     env = env.unwrapped
     epochs, penalties, reward, = 0, 0, 0
     
     done = False
     
     while not done:
      
      if random.uniform(0, 1) < epsilon:
       action = env.action_space.sample() # explore
      
      else:
       action = np.argmax(q_table[state]) # exploit

      next_state, reward, done, info = env.step(action) 
     
      old_value = q_table[state, action]
      next_max = np.max(q_table[next_state])
        
      new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
      q_table[state, action] = new_value
     
      state = next_state

     if i % 1000 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

    print('Training completed')
    
    return q_table

Run the following code to implement your algorithm with pre-defined values:

In [ ]:
env = gym.make('Taxi-v3')
env.reset()
Q_learn_pol = Q_learning_train(env,alpha = 0.2, gamma = 0.95, epsilon = 0.1, episodes = 100000)

Episode: 99000
Training completed


Repeat the experiment: run the algorithm for 1000 times and get the average number of steps to complete the mission. Compare it with the results of the random search approach.

In [ ]:
env = gym.make('Taxi-v3')

 # your code:
average_timesteps = []
average_penalties = []
for _ in range(1000):   
 penalties, reward = 0, 0
 state = env.reset()

 for t in range(10000): # 1000 = maximum number of steps
  action = np.argmax(Q_learn_pol[state])
  state, reward, done, info = env.step(action)
  if reward == -10:
   penalties += 1
  if done:
   #print("Episode finished after {} timesteps".format(t+1))
   #print('number of penalties:', penalties)
   average_timesteps.append(t+1)
   average_penalties.append(penalties)
   break
 env.close()

In [ ]:
def Average(lst):
    return sum(lst) / len(lst)
print('average steps: ', Average(average_timesteps))
print('average penalties: ', Average(average_penalties))